In [1]:
import requests
from bs4 import BeautifulSoup
import time
#기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')

#크롤링 패키지
import requests
from selenium import webdriver

headers = {
    "User-Agent":"각자의 User-Agent를 넣어주자",
    "Accept-Language":"ko-KR,ko"
    }

from selenium import webdriver
driver = webdriver.Chrome("c:/Users/clfrh/chromedriver_win32/chromedriver.exe")
driver.maximize_window()

url = 'https://www.yogiyo.co.kr/'
driver.get(url)

In [2]:
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(2)
    print(location+'으로 위치 설정 완료!')

In [4]:
food_dict = {'프랜차이즈':3, '치킨':4, '피자&양식':5, '중국집':6, 
             '한식':7, '일식&돈까스':8, '족발&보쌈':9, '야식': 10,
             '분식':11, '카페&디저트': 12}

In [5]:
#요기요 카테고리 Element Number Dictionary 정의

def go_to_category(category):
    print(category + '카테고리 페이지 로드중')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(3)
    print(category+'카테고리 페이지 로드 완료')    

In [6]:
def scroll():

    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 웹페이지 맨 아래까지 무한 스크롤
    while True:
        # 스크롤을 화면 가장 아래로 내린다
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
        # 페이지 로딩 대기
        time.sleep(2)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

In [7]:
def get_restaurant_list():
    restaurant_list = []
    restaurants = driver.find_elements_by_css_selector('#content > div.content.ng-scope > div > div > div.restaurant-list > div.col-sm-6.contract')
    print(len(restaurants))
    
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr> td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
        print(restaurant)
    return list(set(restaurant_list))

In [8]:
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        print('1')
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[15]/form/div/input').send_keys(restaurant_name)
        print('1')
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(5)

In [9]:
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(5)

In [10]:
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')

In [11]:
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)

# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll()
            click_more_review()
        except Exception as e:
            pass
    scroll()
    print('모든 리뷰 불러오기 완료!')

In [12]:
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews


In [ ]:
#해당 음식점의 모든 리뷰 객체 리턴

def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews

In [13]:
df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])

for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = {
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass

0it [00:00, ?it/s]


In [14]:
def save_pickle(location, category, yogiyo_df):
    pickle.dump(yogiyo_df, open('./{}_{}_df.pkl'.format(location, category),'wb'))
    print('{} {} pikcle save complete!'.format(location, category))

In [ ]:
#크롤링 메인 함수

def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant', 'UserID', 'Menu', 'Review','Total',
                               'Taste', 'Quantity', 'Delivery', 'Date'])
    
    try:
        set_location(location)
        go_to_category(category)
        
        print('Start [ {} - {} ] Crawling...'.format(location, category))
        
        stretch_list_page()
        restaurant_list = get_restaurant_list()
        
        
        for restaurant_name in restaurant_list:
            
            try:
                
                print('********' + restaurant_name + '(' + str(restaurant_list.index(restaurant_name)+1)
                      + '/' + str(len(restaurant_list)) + '번째) ********')
                
                search_restaurant(restaurant_name)
                go_to_restaurant()
                go_to_review()
                stretch_review_page()
                
                for reivew in tqdm(get_all_review_elements()):
                    
                    try:
                        
                        df.loc[len(df)] = {
                            'Restaurant' : driver.find_element_by_class_name('restaurant_name').text,
                            'UserID' : review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu' : review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review' : review.find_element_by_css_selector('p').text,
                            'Total' : str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste' : review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity' : review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery' : review.find_element_by_css_selector('div:nth-child(2) > div > span.category > sapn:nth-child(9)').text,
                            'Date' : review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,                            
                        }
                    
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
                
            except Exception as e:
                print('*** ' + 'restaurant_name' + ' *** 음식점 페이지 에러')
                go_back_page()
                print(e)
                pass
            
            print('음식점 리스트 페이지로 돌아가는 중')
            go_back_page()
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
    
    print('End of [ {} - {} ] Crawling!'.format(location, category))
    save_pickle(location, category, df)

In [15]:
save_pickle('가톨릭대 성심교정2', '치킨', df)

가톨릭대 성심교정2 치킨 pikcle save complete!


In [18]:
with open('./data/가톨릭대 성심교정2_치킨_df.pkl', 'rb') as f:
    data = pickle.load(f)

In [19]:
data

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
0,오떡후-송내점,lo**님,"오떡후떡볶이 커플세트/1(사이즈 선택(2~3人),구성 선택(반반),떡볶이 선택(매콤...",맵찔이라서 떡볶이시킬때 매우면 어쩌나 걱정하곤했는데 정말 맛있게 잘 먹었습니다 떡도...,5,5,5,5,2일 전
1,오떡후-송내점,fh**님,"오떡후떡볶이 커플세트/1(사이즈 선택(2~3人),구성 선택(떡 많이),떡볶이 선택(...",매운걸 잘 못먹는 편이라 보통맛 순한맛 고민하다가 순한맛 시켰는데 저희한테 아주 딱...,5,5,5,5,2021년 2월 4일
2,오떡후-송내점,jo**님,"매콤까르보나라 오떡후떡볶이/1(사이즈 선택(2~3人),구성 선택(반반),맵기 선택(...",여윽시 오떡후 까르보나라!! 쫀맛탱~~~ 떡이랑 오뎅 둘다 길쭉길쭉해서 먹는데 재밌쪄용!,5,5,5,5,2021년 2월 3일
3,오떡후-송내점,md**님,"오떡후떡볶이 커플세트/1(사이즈 선택(2~3人),구성 선택(반반),떡볶이 선택(매콤...",처음시켰는데 떡이길고 두꺼워서 양이 생각보다많았어요 ㅎㅎ 맛있게 잘먹었습니다!,5,5,5,5,2021년 1월 30일
4,오떡후-송내점,sk**님,"오떡후 순살치킨SET/1(구성 선택(반반),떡볶이 선택(매콤까르보나라 오떡후（순한맛）))",매울줄 알고 까르보나라떡볶이 시켰다가 후회 했네요 ㅋㅋㅋ 담에는 약간 매운거랑 치킨...,5,5,5,5,2021년 1월 25일
...,...,...,...,...,...,...,...,...,...
139,오떡후-송내점,손님,오떡후 커플 SET（오떡후2~3人＋튀김SET＋주먹밥＋계란찜）/1(떡볶이 선택(매콤오...,의외로 배달 빨라서 놀랐어영!!! 맛도 있고 양도 bb 서비스 쿨피스 감사합니당,5,5,5,5,2019년 9월 30일
140,오떡후-송내점,11**님,오떡후 커플 SET（오떡후2~3人＋튀김SET＋주먹밥＋계란찜）/1(떡볶이 선택(매콤오...,Good.........,5,5,5,4,2019년 9월 29일
141,오떡후-송내점,ck**님,"까르보나라 오떡후/1(사이즈 선택(2~3人),구성 선택(어묵많이),토핑 추가 선택(...",양도 엄청많고 맛있었어요~!!,5,5,5,5,2019년 9월 29일
142,오떡후-송내점,ry**님,오떡후 커플 SET（오떡후2~3人＋튀김SET＋주먹밥＋계란찜）/1(떡볶이 선택(매콤오...,떡이랑 오뎅이!! 신기하게 생겼어요 ㅋㅋㅋ 길쭉길쭉~ 제가 국물류 좋아하는데 떡볶이...,5,5,5,5,2019년 9월 28일
